## Definitions

## 1.1 Bellman Equations

**Definition (State)**

The state variable $X_t$ is a vector listing current values of variables relevant to choosing the current action.

**Definition (Value function)**

The value function $v_t(w)$ track the optimal lifetime payoffs from a given state at a given time. It depends only on the parameters (think about the *indirect utility function*). 

**Definition (Bellman equation)**

**Definition (Bellman's Principle of Optimality)**


## 1.2 Stability and Contractions

****

## Structure of a typical dynamic program

For a time period $t<T$, we (objective is to maximize the expected lifetime rewards **EPV**)

- observe the current state $X_t$

- choose an action $A_t$

- receive a reward $R_t(X_t,A_t)$

- update $X_{t+1} = F(X_t, A_t, \xi_{t+1})$

## Finite-Horizon Job Search Problem

This section use a finite-horizon job search problem to illustrate finite-period DP. In particular

- agent begins unemployed at time $t=0$

- receive a new job offer paying wage $W_t$ for all $t = 0,1, 2,\ldots, T$

- Two choices and corresponding rewards
  - *accept* $\implies$ work *permanetly* with wage at the time accepting the offer
  - *reject* $\implies$ receive constant unemployment compensation $c$ for the current period
  
The state variable is wage $W_t\sim \varphi\in \mathcal{D}(W)\,\,iid, W\subset \mathbb{R}_{+}$ finite. Action is to accept or reject the offer $A_{t}=0$ reject, $A_{t} = 1 $ accept.


We can represent this problem using Bellman Equations for each period $t = 0,1,2,\ldots, T$,i.e.,

$$
\begin{align}
v_t(w_t) &= \max\left\{\text{stopping value, continuation value}\right\}\\
&=\max\left\{\sum_{\tau = 1}^{T-t} \beta^\tau w_t, c + \beta \sum_{w'\in W} v_{t+1}(w')\varphi(w')\right\}
\end{align}
$$

We can solve for all $v_t$ by backward induction to calculate the reservation wage at each period. This solves the problem of whether to accept or reject the offer.